
# Building Multi-Agent Systems with Strands Agent Graph / 使用 Strands Agent Graph 构建多代理系统
Multi-agent systems leverage multiple specialized AI agents working together to solve complex problems through coordinated collaboration. Each agent has specific capabilities and roles, connected through explicit communication pathways.

In this lab, you'll learn to build multi-agent systems using the Strands Agent SDK. We'll progress from basic concepts to advanced implementations, exploring different topologies and real-world applications.

**Learning Objectives:**
By the end of this notebook, you'll be able to:
- Understand the three core components of agent graphs (nodes, edges, conditions)
- Send targeted messages between specific agents
- Monitor and control multi-agent networks
- Design specialized agent systems for real-world scenarios

多代理系统利用多个专门的 AI 代理协同工作，通过协调协作解决复杂问题。每个代理都具有特定的能力和角色，通过明确的通信路径连接。

在本实验中，您将学习使用 Strands Agent SDK 构建多代理系统。我们将从基本概念进展到高级实现，探索不同的拓扑结构和实际应用。

**学习目标：**
在本笔记本结束时，您将能够：
- 理解代理图的三个核心组件（节点、边、条件）
- 在特定代理之间发送有针对性的消息
- 监控和控制多代理网络
- 为实际场景设计专门的代理系统


### Importing required packages / 导入必需包

Next we can import the required packages

接下来我们可以导入必需的包

In [ ]:
from strands import Agent

## Understanding Agent Graph Components / 理解代理图组件
An agent graph is a structured network of interconnected AI agents designed to solve complex problems through coordinated collaboration. Each agent represents a specialized node with specific capabilities, and the connections between agents define explicit communication pathways.

代理图是一个结构化的互连 AI 代理网络，旨在通过协调协作解决复杂问题。每个代理代表一个具有特定能力的专门节点，代理之间的连接定义了明确的通信路径。

Before we start building, let's understand the three primary components of an agent graph:

在开始构建之前，让我们了解代理图的三个主要组件：

### 1. Nodes (Agents)
Each node represents an AI agent with:
- **Identity**: Unique identifier within the graph
- **Role**: Specialized function or purpose
- **System Prompt**: Instructions defining the agent's behavior
- **Tools**: Capabilities available to the agent

### 1. 节点（代理）
每个节点代表一个 AI 代理，具有：
- **身份**：图内的唯一标识符
- **角色**：专门功能或目的
- **系统提示**：定义代理行为的指令
- **工具**：代理可用的能力

### 2. Edges (Connections)
Edges define communication pathways with:
- **Direction**: One-way or bidirectional information flow
- **Condition**: Optional function that determines if the edge should be traversed
- **Dependencies**: Define execution order and data flow between nodes

### 2. 边（连接）
边定义通信路径，具有：
- **方向**：单向或双向信息流
- **条件**：确定是否应该遍历边的可选函数
- **依赖关系**：定义节点之间的执行顺序和数据流

### 3. GraphBuilder
The GraphBuilder provides a simple interface for constructing graphs:
- **add_node()**: Add an agent or multi-agent system as a node
- **add_edge()**: Create a dependency between nodes
- **set_entry_point()**: Define starting nodes for execution
- **build()**: Validate and create the Graph instance

### 3. GraphBuilder
GraphBuilder 为构建图提供了简单的接口：
- **add_node()**：将代理或多代理系统添加为节点
- **add_edge()**：在节点之间创建依赖关系
- **set_entry_point()**：定义执行的起始节点
- **build()**：验证并创建 Graph 实例



### Basic processing / 基本处理

Let's start with a simple example of one task processed by two different agents providing an output that will depend on their defined role. Take a look at the execution order of the nodes and also the fact that with STrands SDK you can explicitly get a response only from one single node if needed. Architecture looks as following:

让我们从一个简单的示例开始，一个任务由两个不同的代理处理，提供的输出将取决于它们定义的角色。请注意节点的执行顺序，以及使用 Strands SDK 您可以在需要时明确地仅从一个节点获取响应的事实。架构如下所示：

<div style="text-align:left">
    <img src="images/basic.png" width="55%" />
</div>

In [ ]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

# Create specialized agents
coordinator = Agent(name="coordinator", system_prompt="You are a research team leader coordinating specialists. Provide a short analysis, no need for follow ups")
analyst = Agent(name="data_analyst", system_prompt="You are a data analyst specializing in statistical analysis. Provide a short analysis, no need for follow ups")
domain_expert = Agent(name="domain_expert", system_prompt="You are a domain expert with deep subject knowledge. Provide a short analysis, no need for follow ups")

# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(coordinator, "team_lead")
builder.add_node(analyst, "analyst")
builder.add_node(domain_expert, "expert")

# Add edges (dependencies)
builder.add_edge("team_lead", "analyst")
builder.add_edge("team_lead", "expert")

# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("team_lead")

# Build the graph
graph = builder.build()

#Execute task on newly built graph
result = graph("Analyze the impact of remote work on employee productivity.Provide a short analysis, no need for follow ups")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")


# Get results from specific nodes
print("\n")
print("=============Expert node results only:======================")
print("============================================================")
print(result.results["expert"].result)

### Parallel processing / 并行处理

Now let's create a topology when we will have 2 agents processing the request looking at 2 different aspect  of the problem and have them input into a final agent responsible for summarization and risk calculation based on provided input 

现在让我们创建一个拓扑结构，其中我们将有 2 个代理处理请求，从问题的 2 个不同方面进行分析，并将它们输入到负责根据提供的输入进行总结和风险计算的最终代理中

<div style="text-align:left">
    <img src="images/parallel.png" width="55%" />
</div>

In [ ]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# Create specialized agents

financial_advisor = Agent(name="financial_advisor", system_prompt="You are a financial advisor focused on cost-benefit analysis, budget implications, and ROI calculations. Engage with other experts to build comprehensive financial perspectives.")
technical_architect = Agent(name="technical_architect", system_prompt="You are a technical architect who evaluates feasibility, implementation challenges, and technical risks. Collaborate with other experts to ensure technical viability.")
market_researcher = Agent(name="market_researcher", system_prompt="You are a market researcher who analyzes market conditions, user needs, and competitive landscape. Work with other experts to validate market opportunities.")
risk_analyst = Agent(name="risk_analyst", system_prompt="You are a risk analyst who identifies potential risks, mitigation strategies, and compliance issues. Collaborate with other experts to ensure comprehensive risk assessment.")


# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(financial_advisor, "finance_expert")
builder.add_node(technical_architect, "tech_expert")
builder.add_node(market_researcher, "market_expert")
builder.add_node(risk_analyst, "risk_analyst")

# Add edges (dependencies)
builder.add_edge("finance_expert", "tech_expert")
builder.add_edge("finance_expert", "market_expert")
builder.add_edge("tech_expert", "risk_analyst")
builder.add_edge("market_expert", "risk_analyst")


# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("finance_expert")

# Build the graph
graph = builder.build()

print("============================================================")
print("============================================================")

#Execute task on newly built graph
result = graph("Our company is considering launching a new AI-powered customer service platform. Initial investment is \$2M with projected 3-year ROI of 150%. What's your financial assessment?")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")


# Get results from specific nodes

print("Financial Advisor:")
print("============================================================")
print("============================================================")
print(result.results["finance_expert"].result)
print("\n")

print("Technical Expert:")
print("============================================================")
print("============================================================")
print(result.results["tech_expert"].result)
print("\n")

print("Market Researcher:")
print("============================================================")
print("============================================================")
print(result.results["market_expert"].result)
print("\n")

### Branching with conditions / 带条件的分支

Let's create an agent graph that would classify the request and depending on conditions we define in the code - will route the request either to technical or business agent.

让我们创建一个代理图，它将对请求进行分类，并根据我们在代码中定义的条件——将请求路由到技术或业务代理。

Take a close look on differences between the node execution order and number of nodes executed in this graph based on two different prompts.

仔细观察基于两个不同提示的节点执行顺序和此图中执行的节点数量之间的差异。

<div style="text-align:left">
    <img src="images/conditional.png" width="55%" />
</div>



In [ ]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# Create specialized agents

classifier = Agent(name="classifier", system_prompt="You are an agent responsible for classification of the report request, return only Technical or Business clasification.")
technical_report = Agent(name="technical_expert", system_prompt="You are a technical expert htat focuses on providing short summary from technical perspective")
business_report = Agent(name="business_expert", system_prompt="You are a business expert that focuses on providing short summary from business perspective")

# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(classifier, "classifier")
builder.add_node(technical_report, "technical_report")
builder.add_node(business_report, "business_report")

def is_technical(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "technical" in result_text.lower()

def is_business(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "business" in result_text.lower()

# Add edges (dependencies)
builder.add_edge("classifier", "technical_report", condition=is_technical)
builder.add_edge("classifier", "business_report", condition=is_business)

# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("classifier")

# Build the graph
graph = builder.build()

print("============================================================")
print("============================================================")

#Execute task on newly built graph
result = graph("Provide report on technical aspect of working from home, outline things to consider and key risk factors")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

# Get results from specific nodes

print("Classifier:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

#Execute task on newly built graph
result = graph("Provide report on business impact of working from home, outline things to consider and key risk factors")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

# Get results from specific nodes

print("Classifier:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

## Key Takeaways and Best Practices / 关键要点和最佳实践


### Best Practices:

- **Design for acyclicity:** Ensure your graph has no cycles</p>
- **Use meaningful node IDs:** Choose descriptive names for nodes</p>
- **Validate graph structure:** The builder will check for cycles and validate entry points</p>
- **Handle node failures:** Consider how failures in one node affect the overall workflow</p>
- **Use conditional edges:** For dynamic workflows based on intermediate results</p>
- **Consider parallelism:** Independent branches can execute concurrently</p>
- **Nest multi-agent patterns:** Use Swarms within Graphs for complex workflows</p>
- **Leverage multi-modal inputs:** Use ContentBlocks for rich inputs including images</p>

### 最佳实践：

- **为非循环性设计：** 确保您的图没有循环
- **使用有意义的节点 ID：** 为节点选择描述性名称
- **验证图结构：** 构建器将检查循环并验证入口点
- **处理节点失败：** 考虑一个节点的失败如何影响整体工作流
- **使用条件边：** 用于基于中间结果的动态工作流
- **考虑并行性：** 独立分支可以并发执行
- **嵌套多代理模式：** 在图中使用群体进行复杂工作流
- **利用多模态输入：** 使用 ContentBlocks 进行包括图像在内的丰富输入

## Conclusion / 结论

You've now mastered the fundamentals of building multi-agent systems with Strands Agent Graph! You can create sophisticated networks of specialized AI agents that collaborate to solve complex problems.

The key to successful multi-agent systems is:
- Matching topology to use case
- Defining clear agent roles and responsibilities  
- Establishing proper communication patterns
- Managing resources and cleanup effectively

From here, you can build increasingly sophisticated systems for real-world applications in research, content creation, decision-making, customer service, and beyond.

您现在已经掌握了使用 Strands Agent Graph 构建多代理系统的基础知识！您可以创建复杂的专门 AI 代理网络，它们协作解决复杂问题。

成功的多代理系统的关键是：
- 将拓扑结构与用例匹配
- 定义明确的代理角色和职责
- 建立适当的通信模式
- 有效管理资源和清理

从这里开始，您可以为研究、内容创作、决策制定、客户服务等实际应用构建日益复杂的系统。